<a href="https://colab.research.google.com/github/ShotaSasaki-HCU/Prog2kakushin/blob/main/work2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Numpyの練習
# Improved 2D Phased Array Simulator

import numpy as np

range_x_R = 1 # x軸方向レンジ右(mm)
range_x_L = -1 # x軸方向レンジ左(mm)
range_y_T = 1 # y軸方向レンジ上(mm)
range_y_B = -1 # y軸方向レンジ下(mm)
calc_width = 1 # 計算幅(mm)
tr_num = 5 # 送信素子数（個）
tr_directivity = 95 # 送信素子指向性（度）
tr_clearance = 8 # 送信素子間隔(mm)
beam_direction = np.pi / 2 # 合成波の指向方向(rad)
